# **2. Epoching the data**
---

The goal of this notebook is to epoch the data i.e. :
- Data aligned on **stimulus presentation** (sample_stim) or **response** (sample_resp)
- Chose the neural rhythm : **gamma** (f50f150) or **beta** (f8f24)
- Build a table with anatomical informations (i.e. contact names, parcel names with different atlas)

At the end of this notebook, we should end up with three types of files :
1. Single-subject excel file of length $(n_{trials})$ with time codes for each event (see *1_extract_events*)
2. Single-subject data of shape $(n_{trials}, n_{contacts}, n_{times})$
3. Single-subject excel file of length $(n_{contacts})$  describing single-contact anatomical information

In [1]:
import os
import json
import re
import glob

import numpy as np
import pandas as pd
import xarray as xr

from neo.rawio import ElanRawIO
from neo.io import ElanIO


---

# **Global variables**

In [6]:
################################################################################
# Alignement settings
# -------------------

# Chose to extract data align on :
# - Stimulus presentation : "sample_stim"
# - Subject's response    : "sample_resp"
reference = 'sample_stim'

# Number of time points (before, after) the reference (i.e. stim or resp)
n_samples_before = 128
n_samples_after = 256

# Frequency settings
# ------------------

# Frequency to consider :
# - Gamma : "f50f150"
# - Beta  : "f8f24"
freq = "f50f150"

# amount of temporal smoothing
smoothing = "sm0"

# Folder settings
# ---------------

# Where the data are located
root = '/hpc/brainets/data/db_ebrains/seeg'

# where to save the results
analysis = '/hpc/brainets/data/db_ebrains/analysis'

# events to keep when exporting the data
tr_coords = ['response_time', 'correct', 'difficulty', 'T_orientation']

# Cohort settings
# ---------------
cfg = {
    "I": {
        "s_range": [1, 30],
        "seeg_folder": "seeg"
    },
    "II": {
        "s_range": [31, 50],
        "seeg_folder": "seeg"
    },
    "III": {
        "s_range": [51, 70],
        "seeg_folder": "sEEG"
    }
}

# subject template name
subject = 'HID-Sub-000'
################################################################################

---

# **Global function**
## Function for cleaning contacts name

In [3]:
def clean_contacts(ch_names):
    """Clean contact names.
    
    Rules :
    - Remove empty spaces
    - Replace ' with p (convention for contacts in left / right hemispheres)
    - Lower case contact names
    - Two digit number (e.g. "a2-a1" -> "a02-a01")
    """
    ch_names = [n.split('.')[0].lower().strip().replace(' ', '').replace("'", "p") for n in ch_names]
    # add a 0 for single digit
    for n_i, n in enumerate(ch_names):
        if '-' in n: continue  # skip bipolar cleaning
        letter = re.findall(r'[a-z]*', n)[0]
        number = re.findall(r'\d+', n)[0]
        if len(number) == 1:
            number = '0' + number
        ch_names[n_i] = letter + number
    return np.asarray(ch_names)

## Load the data of a single subject

In [4]:
def load_subject(s_name, coh, seeg_folder, rm_contacts):
    """Load the data of a single subject.
    
    Parameters
    ----------
    suj : subject number
    coh : cohorte number
    seeg_folder : folder name of the seeg data
    rm_contacts : list of contacts to remove
    """
    # _________________________________ EVENTS _________________________________
    # load events
    file_event = os.path.join(analysis, 'events', f'{s_name}.xlsx')
    if not os.path.isfile(file_event):
        print(f"Subject {s_name} has no events. SKIPPED")
        return None, None
    df_events = pd.read_excel(file_event)

    # _________________________________ ANATOMY ________________________________
    # load the anatomy
    folder_anat = os.path.join(root, f'Cohort {coh}', 'anat', s_name)
    if not os.path.isdir(folder_anat):
        print(f"Anatomy folder {folder_anat} doesn't exist. SKIPPED")
        return None, None
    anat_file = glob.glob(f'{folder_anat}/*.csv')
    if not len(anat_file) == 1:
        print(f"Subject {s_name} has no anatomical file. SKIPPED")
        return None, None
    df_anat = pd.read_csv(anat_file[0], skiprows=2, sep='\t')

    # clean anat contacts
    df_anat['contact'] = clean_contacts(list(df_anat['contact']))

    # ___________________________________ DATA _________________________________
    # find the data file
    folder_anat = os.path.join(root, f'Cohort {coh}', seeg_folder, s_name)
    data_file = glob.glob(f'{folder_anat}/*_MCSE_{freq}_*_{smoothing}.eeg')
    pos_file = glob.glob(f'{folder_anat}/*_MCSE_*.pos')
    if (len(data_file) != 1) and (len(pos_file) == 1):
        print(f"Subject {s_name} doesn't have data file or pos file. SKIPPED")
        return None, None

    # load the electrophysiological data
    reader = ElanIO(data_file[0], posfile=pos_file[0])
    reader.parse_header()
    block = reader.read(lazy=False)        # get the first block
    segment = block[0].segments            # get data from first (and only) segment
    signals = segment[0].analogsignals[0]  # get first (multichannel) signal
    data = signals.rescale('V').magnitude.astype(np.float32)
    unit = 'V'
    sfreq = signals.sampling_rate.magnitude

    # get the names from header
    ch_names = reader.header['signal_channels']['name']

    # clean contact names
    ch_names = clean_contacts(ch_names)
    
    # find contacts that are both in the data and in the anat
    ch_keep = []
    for c in ch_names:
        is_in_anat = c in list(df_anat['contact'])
        is_not_bad = c not in rm_contacts
        ch_keep += [is_in_anat and is_not_bad]
    ch_keep = np.asarray(ch_keep)
    if not all(ch_keep):
        print(f"Subject {s_name} has missing / bad channels : {ch_names[~ch_keep]}")
        ch_names = ch_names[ch_keep]
        data = data[:, ch_keep]
    assert data.shape[1] == len(ch_names)

    # reindex according to channel names
    df_anat = df_anat.set_index('contact').loc[ch_names].reset_index()

    # __________________________________ EPOCHS ________________________________
    # extract epochs
    epochs = []
    for s in list(df_events[reference]):
        epochs += [data[s - n_samples_before:s + n_samples_after, :].T]
    epochs = np.stack(epochs, 0)

    # __________________________________ XARRAY ________________________________
    # prepare coordinates
    trials = list(df_events['difficulty'])
    times = (np.arange(0, epochs.shape[-1]) - n_samples_before) / sfreq
    roi = list(df_anat['MarsAtlas'])

    # add additional coordinates
    roi_sup = dict(contacts=('roi', ch_names))
    trials_sup = {k: ('trials', list(df_events[k])) for k in tr_coords}

    # prepare attributes
    attrs = dict(
        sfreq=sfreq, reference=reference, unit=unit, smoothing=smoothing,
        n_samples_before=n_samples_before, n_samples_after=n_samples_after,
        freq=freq, subject=s_name
    )

    # xarray transformation
    epochs = xr.DataArray(epochs, dims=('trials', 'roi', 'times'),
                          coords=(trials, roi, times), name=freq, attrs=attrs)
    epochs = epochs.assign_coords(**roi_sup)
    epochs = epochs.assign_coords(**trials_sup)
    
    return epochs, df_anat

---

# **Extract epochs**

In [7]:
###############################################################################
# list of subjects to ignore and reason
bad_subjects = {
    30: "Anatomical file is not correctly defined",
    38: "Contact names lead to empty array"
}

# bad contacts of individual subject to drop 
bad_contacts = {
    13: ['m02', 'm03'],
    21: ['o12'],
    15: ['sp02'],
    66: ['gp02'],
    50: ['np03', 'np04', 'np08']
}
################################################################################

# define where to save the epochs
save_root = os.path.join(analysis, 'epochs', reference, f"{freq}-{smoothing}")
anat_folder = os.path.join(save_root, 'anat')
data_folder = os.path.join(save_root, 'data')

# create folders (only if don't exist)
if not os.path.isdir(anat_folder): os.makedirs(anat_folder)
if not os.path.isdir(data_folder): os.makedirs(data_folder)

# define how to save the files
anat_file = os.path.join(anat_folder, '%s.xlsx')
_f = f'%s_{freq}-{smoothing}_nbefore-{n_samples_before}_nafter-{n_samples_after}.nc'
data_file = os.path.join(data_folder, _f)


for coh in ['I', 'II', 'III']:
    suj_range = cfg[coh]['s_range']
    seeg_folder = cfg[coh]['seeg_folder']
    
    for suj in range(suj_range[0], suj_range[1] + 1):   
        if suj in bad_subjects.keys():
            print(f"Subject {suj} SKIPPED :  {bad_subjects[suj]}")
            continue
        
        # build subject name
        id_suj = str(suj)
        s_name = subject[0:-len(id_suj)] + id_suj
        
        # if already computed, skip
        if os.path.isfile(data_file % s_name):
            print(f"Subject {s_name} already computed. SKIPPED")
            continue
        
        # get bad contacts (if possible)
        rm_contacts = bad_contacts.get(suj, [])

        # extract epochs and clean anatomy
        print(f"Cohort {coh} : {suj}", end='\r')
        epochs, df_anat = load_subject(s_name, coh, seeg_folder, rm_contacts)
        
        if epochs is None: continue
        
        # save files
        epochs.to_netcdf(data_file % s_name)
        df_anat.to_excel(anat_file % s_name)


Subject HID-Sub-001 already computed. SKIPPED
Subject HID-Sub-002 already computed. SKIPPED
Subject HID-Sub-003 already computed. SKIPPED
Subject HID-Sub-004 already computed. SKIPPED
Subject HID-Sub-005 already computed. SKIPPED
Subject HID-Sub-006 has no events. SKIPPED
Subject HID-Sub-007 has no events. SKIPPED
Subject HID-Sub-008 already computed. SKIPPED
Subject HID-Sub-009 already computed. SKIPPED
Subject HID-Sub-010 already computed. SKIPPED
Subject HID-Sub-011 already computed. SKIPPED
Subject HID-Sub-012 already computed. SKIPPED
Subject HID-Sub-013 already computed. SKIPPED
Subject HID-Sub-014 already computed. SKIPPED
Subject HID-Sub-015 already computed. SKIPPED
Subject HID-Sub-016 already computed. SKIPPED
Subject HID-Sub-017 already computed. SKIPPED
Subject HID-Sub-018 already computed. SKIPPED
Subject HID-Sub-019 already computed. SKIPPED
Subject HID-Sub-020 already computed. SKIPPED
Subject HID-Sub-021 already computed. SKIPPED
Subject HID-Sub-022 already computed. SK